# <font color='blue'>THÔNG TIN NHÓM</font>

- STT: 5
- Tên Nhóm: GoGo
- Nhóm 4 thành viên:
    + 18120507 Trương Công Phu
    + 18120517 Nguyễn Công Bình Phương
    + 18120532 Nguyễn Hoàng Sang
    + 18120626 Đặng Quang Trường

# <font color='blue'>PHÂN TÍCH</font>

## 1. THÔNG TIN DỮ LIỆU

Đây là tập dữ liệu về tai nạn xe hơi trên toàn quốc, bao gồm 49 tiểu bang của Hoa Kỳ. Dữ liệu tai nạn được thu thập từ tháng 2 năm 2016 đến tháng 12 năm 2020, sử dụng nhiều API cung cấp dữ liệu sự cố (hoặc sự kiện) giao thông trực tuyến. Các API này truyền phát dữ liệu giao thông được thu thập bởi nhiều thực thể, chẳng hạn như bộ giao thông vận tải Hoa Kỳ và tiểu bang, cơ quan thực thi pháp luật, camera giao thông và cảm biến giao thông trong mạng lưới đường bộ. Hiện tại, có khoảng 3 triệu hồ sơ tai nạn trong cơ sở dữ liệu này
- Nguồn dữ liệu: https://www.kaggle.com/sobhanmoosavi/us-accidents
- Trích dẫn:
    + Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.

    + Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

## 2. TÌM HIỂU VÀ PHÂN TÍCH DỮ LIỆU

### import necessary library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima
from pmdarima.arima.utils import nsdiffs
import datetime
import warnings
# plot
import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True) 
pd.set_option('display.max_columns', 49)
%matplotlib inline
warnings.filterwarnings("ignore")
# if you haven't installed libray above, can use:
# !pip install pmdarima
# !pip install plotly

### Đọc dữ liệu và khám phá sơ lược

In [ ]:
# read_data
df = pd.read_csv('US_Accidents_Dec20_Updated.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
df.head()

In [ ]:
df.info()

#### Xem mô tả dữ liệu

In [ ]:
# description of columns
f_description = open('description.txt', 'r')
content = f_description.readlines()
lines = list(filter(lambda x: x != '\n', content))
descriptions = {line[:line.find(':')] : line[line.find(':') + 2:-1] for line in lines}
descriptions

#### Kiểm tra các cột dữ liệu thiếu

Xem phần trăm thiếu dữ liệu của các cột

In [ ]:
# ratio of missing data
df_ratio_null = df.isnull().sum() / df.shape[0] * 100
df_ratio_null = df_ratio_null[df_ratio_null > 0]
df_ratio_null

* Nhận xét: 
  + Cột Number thiếu đến 65% dữ liệu, ý nghĩa của cột này là số đường nhưng cũng không quan trọng lắm vì ta đã có tên đường (Street) làm đại diện, ta sẽ bỏ đi cột này
  + Cột Wind_Chill(F) và cột Precipitation(in) thiếu dữ liệu khá nhiều(40,7% và 44,7%). Ta kiểm tra xem có phải các dữ liệu thiếu thì ý nghĩa là bằng 0 hay không. Tức là kiểm tra xem ngoài những giá trị Nan thì có giá trị bằng 0 hay không.Nếu không có thì ta thay các giá trị Nan bằng 0, còn nếu có thì ta sẽ bỏ luôn 2 cột này.
  + Các cột còn lại có số lượng giá trị không nhiều lắm. Ta sẽ xử lí riêng khi phân tích từng thuộc tính.

In [ ]:
# Check column `Wind_Chill(F)` and `Precipitation(in)`
print('number of value "0" in Precipitation {}'.format((df['Precipitation(in)'] == 0).sum()))
print('number of value "Nan" in Precipitation {}'.format((df['Precipitation(in)'].isnull().sum())))
print('number of value greater than "0" in Precipitation {}'.format((df['Precipitation(in)'] > 0).sum()))
print('---------------------------------------')
print('number of value "0" in Precipitation {}'.format((df['Wind_Chill(F)'] == 0).sum()))
print('number of value "Nan" in Precipitation {}'.format((df['Wind_Chill(F)'].isnull().sum())))
print('number of value greater than "0" in Precipitation {}'.format((df['Wind_Chill(F)'] > 0).sum()))

=> Bỏ 2 cột này đi

In [ ]:
df.drop(['Number', 'Precipitation(in)', 'Wind_Chill(F)'], axis=1, inplace=True)
df.columns

### Đầu tiên ta xem sơ lược qua các Bang(State), Thành phố (City) có số lượng tai nạn nhiều nhất

In [ ]:
count_accident_top20_state = df.groupby('State')['State'].count().sort_values(ascending=False)[0:20]
count_accident_top20_city = df.groupby('City')['City'].count().sort_values(ascending=False)[0:20]

fig, axes = plt.subplots(1,2, figsize=(20,10))
sns.barplot(y = count_accident_top20_state.index, x = count_accident_top20_state.values, ax=axes[0])
axes[0].set_title(f'Top number of accident by State')

sns.barplot(y = count_accident_top20_city.index, x = count_accident_top20_city.values, ax=axes[1])
axes[1].set_title(f'Top number of accident by City')
plt.show()

<b>Cùng xem trên graph Map </b>

- <font color='red'>POPULATION</font>

In [ ]:
 '''ref: https://plotly.com/python/bubble-maps/ '''
df_pop = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df_pop.head()
df_pop['name'] = df_pop['name'].str.strip()
df_pop['text'] = df_pop['name'] + '<br>Population ' + (df_pop['pop']/1e6).astype(str)+' million'
limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = 5000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_pop[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

**note**: Mặc dù là dữ liệu 2014, thời gian không hợp với tập dữ liệu ta đang xét. Nhưng ta có thể dùng để minh họa tìm ra insight với một độ tương quan chính xác nhất định

- <font color='red'>TOTAL ACCIDENT</font>

In [ ]:
df_state = df['State'].value_counts()

fig = go.Figure(data=go.Choropleth(
    locations=df_state.index, # Spatial coordinates
    z = df_state.values.astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Sum of accident",
))

fig.update_layout(
    title_text = 'Total Accident of State in USE',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

* Nhận xét: 
    + Tiểu bang California là tiểu bang có số vụ tai nạn cao nhất cao hơn rất nhiều so với top 2 - 5 phía dưới, cao gấp hơn 2 lần. Nguyên nhân đầu tiên chính là tiểu bang này là tiểu bang có dân số nhiều nhất nước Mỹ.
    + 3 thành phố  có tai nạn nhiều nhất là: Los Angeles, Houston, Charlotte. Đặc điểm ở 3 thành phố này cũng đều nằm trong top các thành phố có mật độ dân số cao (Số liệu theo wikipedia).
    + Ta thử kiếm tra các thành phố thuộc bang California xem:

In [ ]:
CA_df = df[df['State'] == 'CA']['City'].value_counts().sort_values(ascending=False).head(10)
plot = sns.barplot(y=CA_df.index, x=CA_df.values)
plot.set_xlabel('City')
plot.set_ylabel('Records')
plot.set_title('Top 10 City có số Records lớn nhất tiểu bang California')

* Nhận xét: thành phố có số Records cao nhất nước nằm ở tiểu bang California. Chính điều này làm cho số Records ở bang này rất cao.

### Các đoạn đường hay xảy ra tai nạn ở các thành phố có có số lượng tai nạn cao

In [ ]:
#Get top 6 City
df_top_6_city = df.groupby('City')['City'].count().sort_values(ascending=False)[0:6]
top_6_city = df_top_6_city.index

fig, axes = plt.subplots(3,2, figsize=(30,20))
index = 0
for i in range(3):
    for j in range(2):
        top_street = df[df['City'] == top_6_city[index]].groupby('Street')['Street'].count().sort_values(ascending=False)[0:10]
        sns.barplot(y=top_street.index, x=top_street.values, ax=axes[i][j])
        axes[i][j].set_title(f'Top 10 street with highest number of accidents in {top_6_city[index]} City')
        index += 1

**Ta thử  cùng xem qua thành phố Los Angeles có số lượng tai nạn cao nhất trên bản đồ thử xem**

In [ ]:
import plotly.express as px
df['accident'] = 1
fig = px.density_mapbox(df[df['City'] == 'Los Angeles'], lat='Start_Lat', lon='Start_Lng', z='accident', hover_name='Street', radius=5,
                        center=dict(lat=34.103172, lon=-118.249969), zoom=12,
                        mapbox_style="open-street-map", height=900)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

* Nhận xét: Tai nạn hay xảy ra ở các đoạn cua, các giao lộ nhiều làn giao nhau.

### Diễn biến số vụ tai nạn qua từng năm.

In [ ]:
# Preprocessing Date columns
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['year']       = df['Start_Time'].dt.strftime("%Y")
df['month']      = df['Start_Time'].dt.strftime("%m")
df['day']        = df['Start_Time'].dt.strftime("%d")
df['week']       = df['Start_Time'].dt.strftime("%a")
df['hour']       = df['Start_Time'].dt.strftime("%H")
df['Week']       = df['Start_Time'].dt.strftime("%W")

In [ ]:
# Visualize
f, ax = plt.subplots(2,2, figsize = (18,10))
f.suptitle('Thống kê số vụ tai nạn theo thời gian', fontsize = 22)

count_year = df['year'].value_counts().sort_index()
sns.barplot(x=count_year.index, y = count_year.values, ax = ax[0][0])
plt.sca(ax[0][0])
plt.xlabel('year')

count_month = df['month'].value_counts().sort_index()
sns.barplot(x=count_month.index, y = count_month.values, ax = ax[0][1], label='month')
plt.sca(ax[0][1])
plt.xlabel('month')

count_week = df['week'].value_counts()
day_map = {'Mon':0 , 'Tue':1 , 'Wed':2 , "Thu":3 , 'Fri':4 , "Sat": 5 , 'Sun':6}
sns.barplot(y = count_week.values, x = count_week.index.map(day_map), ax = ax[1][0])
plt.sca(ax[1][0])
plt.xticks(np.arange(7), day_map.keys())
plt.xlabel('day of week')

count_hour = df['hour'].value_counts().sort_index()
sns.barplot(x=count_hour.index, y = count_hour.values, ax = ax[1][1])
plt.sca(ax[1][1])
plt.xlabel('hour of day')

* Nhận xét: 
    + Tình hình tai nạn xảy ra tăng dần theo năm, đặc biệt là năm 2020 - tăng rất nhanh so với năm 2019(gần gấp đôi).
    + Tai nạn tăng nhanh những tháng cuối năm.
    + Tai nạn thường xảy ra vào các giờ cao điểm như 7-8, 15-17.
    + Tai nạn giảm vào cuối tuần.

### Tìm hiểu về mức độ nghiêm trọng

`Severity: Shows the severity of the accident, a number between 1 and 4, where 1 indicates the least impact on traffic (i.e., short delay as a result of the accident) and 4 indicates a significant impact on traffic (i.e., long delay). Note that severity reported by different sources may differ in their underlying impact on traffic, so please separate data from different sources when doing severity-based analysis.`

* Nhận xét: Theo như file mô tả về thuộc tính Severity, số liệu có vẻ sẽ không thống nhất từ nhiều nguồn. Vậy nên ở thuộc tính này ta sẽ xem xét dựa trên một bộ phận cụ thể.

#### Thống kê mức độ nghiêm trọng (Severity)

In [ ]:
# check feature Severity
count_severity_df = pd.DataFrame(df['Severity'].value_counts())
count_severity_df
plt.figure(figsize=(12,8))
g = sns.barplot(x=count_severity_df.index, y=count_severity_df.Severity)
g.set_xlabel('Severity')
g.set_ylabel('Sum of Severity')
for index, row in count_severity_df.iterrows():
     g.text(index - 1,row.Severity*1.01, round(row.Severity,2), color='black', ha="center")
plt.show()

Ta giả định rằng mức độ tai nạn cao là ở mức `Severity` >= 3. Ta xem thử tần suất mức độ nghiệm trọng cao xảy ra (Severity >= 3) trong 20 group(City, State, ...) có số lượng tai nạn cao nhất như thế nào.
Công thức tính tần suất nghiêm trọng cao: 

$$ \frac{count(Severity >= 3)}{ count(Severity)} $$

#### Thống kê mức độ nghiêm trọng (Severity) theo Bang và Thành Phố

In [ ]:
def get_probility_of_Severity(feature): 
    # Số lượng xảy ra tai nạn ở mỗi "group" theo feature
    count_accident_top20 = df.groupby(feature)[feature].count().sort_values(ascending=False)[0:20]
    df_top_20 = df[df[feature].apply(lambda x: True if x in count_accident_top20.index else False)]
    df_hard_severity = df_top_20.groupby(feature)['Severity'].apply(lambda x: sum([a for a in x if a >= 3]))
    df_severity = df_top_20.groupby(feature)['Severity'].sum()
    df_probility = df_hard_severity / df_severity * 100
    df_probility = df_probility.sort_values(ascending=False)
    fig, axes = plt.subplots(1,2, figsize=(20,10))
    sns.barplot(y = df_probility.index, x = df_probility.values, ax=axes[0])
    axes[0].set_title(f'Top probility of hard severity by {feature}')
    
    sns.barplot(y = count_accident_top20.index, x = count_accident_top20.values, ax=axes[1])
    axes[1].set_title(f'Top number of accident by {feature}')

In [ ]:
get_probility_of_Severity('State')

In [ ]:
get_probility_of_Severity('City')

* Ta sẽ chọn thành phố Los Angeles để tìm hiểu về thuộc tính Severity

#### Heatmap about severity of Los Angeles city

In [ ]:
import plotly.express as px
fig = px.density_mapbox(df[df['City'] == 'Los Angeles'], lat='Start_Lat', lon='Start_Lng', z='Severity', hover_name='Street', radius=5,
                        center=dict(lat=34.103172, lon=-118.249969), zoom=12,
                        mapbox_style="open-street-map", height=900)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#### Mức độ tai nạn giao thông theo ngày trong tuần

In [ ]:
df_LA = df[df['City'] == 'Los Angeles']
df_hard_severity = df_LA.groupby('week')['Severity'].apply(lambda x: sum([a for a in x if a >= 3]))
df_severity = df_LA.groupby('week')['Severity'].sum()

plt.figure(figsize=(12,8))
sns.barplot(y = df_severity.values, x = df_severity.index.map(day_map), 
            color='orange', label='Sum Severity')
sns.barplot(y = df_hard_severity.values, x = df_hard_severity.index.map(day_map), 
            color='r', label='Sum Hard Severity')
plt.xticks(np.arange(7), day_map.keys())
plt.legend()
plt.show()

#### Mức độ tai nạn giao thông theo giờ trong ngày, tháng trong năm và theo từng năm.

In [ ]:
features = ['hour', 'month', 'year']

fig, axes = plt.subplots(3,1, figsize=(10,20))
for i, feature in enumerate(features):
    df_hard_severity = df_LA.groupby(feature)['Severity'].apply(lambda x: sum([a for a in x if a >= 3])).sort_index()
    df_severity = df_LA.groupby(feature)['Severity'].sum().sort_index()
    
    plt.figure(figsize=(12,8))
    sns.barplot(y = df_severity.values, x = df_severity.index, 
                color='orange', label='Sum Severity', ax=axes[i])
    sns.barplot(y = df_hard_severity.values, x = df_hard_severity.index, 
                color='r', label='Sum Hard Severity', ax=axes[i])
    axes[i].legend()

plt.show()

* Nhận xét: Mức độ nghiêm trọng cao có xu hướng giảm theo từng năm mặc dù số tai nạn tăng theo từng năm

### TÌM HIỂU TAI NẠN THEO CÁC ĐIỀU KIỆN GIAO THÔNG

- Theo như file description các cột về điều kiện giao thông ở gần vị trí tai nạn là các cột: `Amenity, Bump, Crossing, Give_Way, Junction, No_Exit, Railway, Roundabout, Station, Stop, Traffic_Calming, Traffic_Signal, Turning_Loop`
- Các cột này có kiểu `bool`, nhận giá trị `True` nghĩa là có xuất hiện điều kiện, `False` thì ngược lại
- Đặc biệt các cột này không thiếu dữ liệu
=> Ta cùng tìm hiểu các cột này

In [ ]:
condition_traffic_df = df.select_dtypes(include='bool')
condition_columns = condition_traffic_df.columns
condition_columns

In [ ]:
df_bool = pd.DataFrame(df[condition_columns].sum(), columns=['True'])
df_bool['False'] = df.shape[0] - df_bool['True']
df_bool = df_bool.T 
df_bool.style.background_gradient(cmap='Wistia',subset=["Amenity"])\
            .background_gradient(cmap='Blues',subset=["Bump"])\
            .background_gradient(cmap='Greens',subset=["Crossing"])\
            .background_gradient(cmap='Greys',subset=["Give_Way"])\
            .background_gradient(cmap='Oranges',subset=["Junction"])\
            .background_gradient(cmap='Reds',subset=["No_Exit"])\
            .background_gradient(cmap='YlOrBr',subset=["Railway"])\
            .background_gradient(cmap='OrRd',subset=["Roundabout"])\
            .background_gradient(cmap='PuRd',subset=["Station"])\
            .background_gradient(cmap='GnBu',subset=["Stop"])\
            .background_gradient(cmap='PuBu',subset=["Traffic_Calming"])\
            .background_gradient(cmap='PuBuGn',subset=["Traffic_Signal"])\
            .background_gradient(cmap='YlGnBu',subset=["Turning_Loop"])

 * Nhận xét: Quan sát heatmap trên, ta nhận thấy rằng các thuộc tính xét ở bảng trên đa số nhận giá trị False, tức là tai nạn thường xảy ra khi thiếu các điệu kiện trên. Theo file mô tả, ý nghĩa của các thuộc tính trên như sau:
    + Amenity: Cho biết có sự hiện diện của một tòa nhà họặc địa điểm nào ở gần không.
    + Bump: Cho biết có sự hiện của gờ giảm tốc ở gần hay không
    + Crossing: Cho biết có xuất hiện nơi có hai đường bộ, hai đường sắt hoặc đường bộ và đường sắt giao nhau ở gần hay không
    + Give_Way: Cho biết có các give_way(nơi nhường đường, biển báo nhường đường) ở gần không
    + Junction: Cho biết có giao lộ ở gần hay không 
    + No_Exit: Cho biết có biển báo no_exit ở gần hay không
    + Railway: Cho biết có đường sắt ở gần hay không
    + Roundabout: Cho biết có bùng binh ở gần hay không
    + Station: Cho biết có các trạm ga tàu ở gần hay không
    + Stop: Cho biết có các biển báo dừng ở gần hay không
    + Traffic_Calming: Cho biết có các biện pháp làm chậm giao thông ở gần hay không
    + Traffic_Signal: Cho biết có các tín hiểu, đèn giao thông ở gần hay không
    + Turning_Loop: Cho biết có vòng xoay ở gần hay không

#### Cùng xem quan hệ corelation của các thuộc tính này. Tuy nhiên ta xe đảo lại 2 giá trị (True, False), tức là True thành False, False thành True. Điều này có ý nghĩa rằng quan hệ giữa sự thiếu các điều kiện giao thông (giá trị bằng True tức là điều kiện đó bị thiếu)

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap((-condition_traffic_df).corr(), annot=True, cbar=True, fmt='.2f', center=0, square=True)

Ta thấy có 2 cặp có độ tương quan cao là (Traffic_Calming, Bump) và (Traffic_Signal, Crossing). Thật ra điều này cũng dễ hiểu bởi vì Trafic_Calming và Bump đều mang ý nghĩa giảm tốc độ. Còn Traffic_Signal và Crossing thì thường là nơi nào có tín hiệu đèn giao thông thì thường có vạch qua đường ở đó)

### ẢNH HƯỞNG CỦA ĐIỆU KIỆN THỜI TIẾT ĐỐI VỚI TAI NẠN GIAO THÔNG

In [ ]:
# columns weather
weather_columns = ['Temperature(F)', 'Humidity(%)', 
                  'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 
                  'Wind_Speed(mph)','Weather_Condition']
weather_df = df[weather_columns]
weather_df.head()

In [ ]:
weather_df.info()

In [ ]:
weather_df.isnull().sum()/weather_df.shape[0]

Trừ cột `Wind_Speed` thì các cột còn lại thiếu dữ liệu khá ít. Ta có thể xóa các dòng thiếu. Cùng xem cột `Wind_speed` để tìm ra cách xử lí hợp lí

In [ ]:
weather_df['Wind_Speed(mph)']

In [ ]:
sns.histplot(weather_df['Wind_Speed(mph)'].dropna().values, kde=True)

dữ liệu phân bố chủ yếu ở phần đầu

In [ ]:
weather_df['Wind_Speed(mph)'].describe()

Ta thấy được rằng dữ liệu của `Wind_Speed` chủ yếu phân bố ở đoạn [4.6, 10.04]. Ta sẽ điền giá trị bị thiếu bằng giá trị trung bình $\frac{4.6 + 10.04}{2} =  7.32$

In [ ]:
weather_df['Wind_Speed(mph)'].fillna(7.32, inplace=True)
weather_df['Wind_Speed(mph)'].isnull().sum()

In [ ]:
# drop row missing value
weather_df.dropna(inplace=True)

#Distributing numerical columns
number_col = weather_df.select_dtypes(include='number').columns
fig, axes = plt.subplots(2,2, figsize=(20,20))
index = 0
for i in range(2):
    for j in range(2):
        sns.histplot(data=weather_df, x=number_col[index], kde=True, ax=axes[i][j])
        index += 1

* Nhận xét
    - Temperature(F): tai nạn xảy ra nhiều ở nhiệt độ trong đoạn [50, 70] (F) (Mức trung bình)
    - Humidity(%): phân phối khá đều, chỉ lệch ở mức độ ẩm 90% và khoảng 100% (hai điểm này tai nạn khá cao)
    - Pressure(in): Tai nạn xảy ra nhiều ở mức áp suất không khí ở khoảng 30in. Có thể cho rằng áp suất không khí không ảnh hưởng lắm vì giá trị xấp xỉ 30in là mức tiêu chuẩn ở gần mặt đất
    - Visibility(mi): Tai nạn xảy ra nhiều ở mức Visibility (khả năng nhìn thấy tính bằng m) khá thấp (phân phối lệch hẳn qua bê trái).

In [ ]:
# caterogical columns
# Weather
weather_df['Weather_Condition'].value_counts()

In [ ]:
(weather_df['Weather_Condition'].value_counts() > 3000).sum()

số lượng nhãn của Weather_Condition khá nhiều (126 nhãn), trong đó chỉ có 22 nhãn là xuất hiện trên 3000 lần (trong 2801332 dòng dữ liệu). Ta sẽ thống kê 22 nhãn xuất hiện cao nhất.

In [ ]:
top_22 = weather_df['Weather_Condition'].value_counts().sort_values(ascending=False)[0:22]
plt.figure(figsize=(12,8))
sns.barplot(y=top_22.index, x=top_22.values)

* Nhận xét: Nhìn biểu đồ trên có vẻ như thời tiết đẹp thì tai nạn nhiều. Có thể suy ra rằng vào thời tiết đẹp thì mọi người ra đường nhiều nên tỉ lệ tai nạn cao

In [ ]:
#Wind Direction
wind_direction_count = weather_df['Wind_Direction'].value_counts()
wind_direction_count

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(y = wind_direction_count.index, x = wind_direction_count.values)

* Nhận xét: có 2 nhãn `CALM` và `calm` chiếm cao nhất, thật ra 2 nhãn này là như nhau, tức lấy tổng lại có hơn 500 000 nhãn `calm`. Tai nạn xảy ra nhiều lúc trời lặng gió`

### Phân tích time series 

#### Lấy dữ liệu là ngày xảy ra tai nạn (thuộc tính `Start_Time`) để dự đoán tình hình tai nạn của những năm tiếp theo.

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
day_time = pd.to_datetime(df['Start_Time'].dt.strftime("%Y-%m-%d"))

y_16_20 = day_time.value_counts().sort_index()
plt.figure(figsize=(16,5), dpi = 120)
plt.plot(y_16_20.index, y_16_20)
plt.title("Diễn biến tai nạn từ năm 2016 - 2020", size =22)
plt.show()

In [ ]:
result = seasonal_decompose(pd.DataFrame(y_16_20), model='multiplicative',period = 3)
plt.rcParams.update({'figure.figsize': (12,10)})
result.plot().suptitle('2016-2020', fontsize=22)

* Nhận xét: Nhận thấy `trend` và `seasonal` của thời gian từ năm 2016 đến năm 2020 không được thể hiện rõ ràng. Khó để dự đoán tình hình tai nạn của các năm tiếp theo.
<b><i> ==> Dự đoán theo từng năm</i> </b> 

# Tìm ra model tốt nhất với tập train đầu vào và chu kì của mùa vụ là m
    + auto_arima sẽ chị ra các giá trị p,q để có được mô hình tốt nhất với tiêu chí là AIC
    (Mô hình có AIC nhỏ hơn nói chung sẽ tốt hơn)
    + Sau khi đã tìm ra được mô hình ARIMA tốt nhất. Chúng ta sẽ dự báo cho khoảng thời gian tiếp theo.
    Dự báo cho chuỗi thời gian khá đặc thù và khác biệt so với các lớp mô hình dự báo khác vì giá trị time step 
    liền trước sẽ được sử dụng để dự báo cho time step liền sau.
##### Sử dụng auto_arima để tìm ra model tốt nhất với các siêu tham số như sau:
    + Các giá trị p, q sẽ được chạy từ [start_p,max_p], [start_q,max_q]
    + m : là giá trị mùa vụ( nghĩa là cứ sau m thì sẽ hình thành một chu kì mới).
    + seasonal = True : để chỉ arima sử dung dự đoán có tính mùa vụ.
    + Chọn d=0
    + Chọn D từ model nsdiffs của arima
    + Phương pháp điều chỉnh mô hình stepwise.

In [ ]:
def best_model(train,m):
    D = nsdiffs(train,m=m,max_D = 12, test= 'ocsb')
    model_sarima = auto_arima(train, start_p=0, start_q=0,
                           max_p=7, max_q=7, m=m,
                           start_P=0, seasonal=True,
                           d=0, D=D, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    return model_sarima

In [ ]:
# Độ lỗi
def _measure_metric(y, y_pred):
    e = y-y_pred
    mse=np.mean(e**2)
    rmse=np.sqrt(mse)
    mae=np.mean(np.abs(e))
    mape=np.mean(np.abs(e/y))*100

    print('Mean Square Error: {}'.format(mse))
    print('Root Mean Square Error: {}'.format(rmse))
    print('Mean Absolute Error: {}'.format(mae))
    print('Mean Absolute Percentage Error: {}'.format(mape))
    return mse, rmse, mae, mape

def draw(f, fitted):
    plt.figure(figsize=(20,6))
    plt.plot(f, label='Actual')
    plt.plot(fitted_seri, color='red', linestyle='--', label = 'Forecast')
    plt.legend()
    plt.show()

#### Phân tích các yếu tố xu hướng (trend), mùa vụ (seasonal), phần dư (residual) qua các năm

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
for i in range(5):
    f = day_time[(day_time >= datetime.datetime(2016+i,1,1)) & (day_time <= datetime.datetime(2016+i,12,31))].value_counts().sort_index()
    result = seasonal_decompose(pd.DataFrame(f), model='multiplicative',period = 3)
    plt.rcParams.update({'figure.figsize': (12,10)})
    result.plot().suptitle('%s'%(str(2016+i)), fontsize=22)
plt.show()

* Nhận xét: Các năm đều có xu hướng và mùa vụ của tai nạn: ==> có thể dự đoán tương lai.

#### Dự đoán:

##### Theo năm: Dùng dữ liệu trong 1 năm với tập train là từ đầu năm đến tháng 11 và tập test là tháng 12

##### - Năm 2016

In [ ]:
# Năm 2016
f = day_time[(day_time >= datetime.datetime(2016,1,1)) & (day_time <= datetime.datetime(2016,12,31))].value_counts().sort_index()
n_pred_perious = 30
train = f[:-n_pred_perious]
test = f[-n_pred_perious:]

- Vẽ các biểu đồ acf và pacf để tìm giá trị mùa vụ:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (20,5))
plot_acf(f, ax = axes[0])
plot_pacf(f, ax = axes[1])
plt.show()

- Chọn m = 8

In [ ]:
model_sarima = best_model(train,m = 8)
fitted, confint = model_sarima.predict(n_periods=n_pred_perious, return_conf_int=True)
date = pd.date_range(train.index[-1], periods=n_pred_perious, freq='D')
fitted_seri = pd.Series(fitted, index=date)

In [ ]:
# độ lỗi
_measure_metric(test, fitted_seri)

In [ ]:
draw(f, fitted_seri)

##### - Năm 2017

In [ ]:
f = day_time[(day_time >= datetime.datetime(2017,1,1)) & (day_time <= datetime.datetime(2017,12,31))].value_counts().sort_index()
n_pred_perious = 30
train = f[:-n_pred_perious]
test = f[-n_pred_perious:]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (20,5))
plot_acf(f, ax = axes[0])
plot_pacf(f, ax = axes[1])
plt.show()

- Chọn: m = 8

In [ ]:
model_sarima = best_model(train,m = 8)
fitted, confint = model_sarima.predict(n_periods=n_pred_perious, return_conf_int=True)
date = pd.date_range(train.index[-1], periods=n_pred_perious, freq='D')
fitted_seri = pd.Series(fitted, index=date)

In [ ]:
_measure_metric(test,fitted_seri)

In [ ]:
draw(f, fitted_seri)

##### Dùng dữ liệu năm 2019 và 2020 để dự đoán 2 tháng đầu năm 2021.

In [ ]:
train = day_time[(day_time >= datetime.datetime(2019,1,1)) 
                 & (day_time <= datetime.datetime(2020,12,31))]\
            .value_counts()\
            .sort_index()

In [ ]:
plot_pacf(train)
plt.show()

Nhìn vào biểu đồ PACF có thể lựa chọn giá trị mùa vụ là 8( vì các giá trị PACF tăng đột biến ở các độ trễ 8, 15 , 22)

In [ ]:
# Predict:
train = day_time[(day_time >= datetime.datetime(2019,1,1)) & (day_time <= datetime.datetime(2020,12,31))].value_counts().sort_index()
n_pred_perious = 60

model_sarima = best_model(train,m = 8)
fitted, confint = model_sarima.predict(n_periods=n_pred_perious, return_conf_int=True)
date = pd.date_range(train.index[-1], periods=n_pred_perious, freq='D')
fitted_seri = pd.Series(fitted, index=date)

In [ ]:
# Predict model
lower = confint[:, 0]
upper = confint[:, 1]

plt.figure(figsize=(16, 8))
plt.plot(train, label='Actual')
plt.plot(fitted_seri, color='red', label = 'Forecast')
plt.fill_between(date, lower, upper, color='k', alpha=0.15)
plt.legend()
plt.title('SARIMA regression model forecast for 60 next days')
plt.show()

#### Dự đoán cho năm 2021:

In [ ]:
train = day_time.value_counts().sort_index()

In [ ]:
f, axes = plt.subplots(1,2,figsize = (20,6))
plot_acf(train, ax = axes[0])
plot_pacf(train, ax = axes[1])
plt.show()

* chọn m = 8

In [ ]:
n_pred_perious = 365
model_sarima = best_model(train,m = 8)
fitted, confint = model_sarima.predict(n_periods=n_pred_perious, return_conf_int=True)
date = pd.date_range(train.index[-1], periods=n_pred_perious, freq='D')
fitted_seri = pd.Series(fitted, index=date)

In [ ]:
# Predict model
lower = confint[:, 0]
upper = confint[:, 1]

plt.figure(figsize=(16, 8))
plt.plot(train, label='Actual')
plt.plot(fitted_seri, color='red', label = 'Forecast')
plt.fill_between(date, lower, upper, color='k', alpha=0.15)
plt.legend()
plt.title('SARIMA regression model forecast for 2021')
plt.show()

## TỔNG KẾT

Các kết luận rút ra được từ việc phân tích trên:
1.  + Tiểu bang California là tiểu bang có số vụ tai nạn cao nhất cao hơn rất nhiều so với top 2 - 5 phía dưới, cao gấp hơn 2 lần. Nguyên nhân đầu tiên chính là tiểu bang này là tiểu bang có dân số nhiều nhất nước Mỹ.
    + 3 thành phố  có tai nạn nhiều nhất là: Los Angeles, Houston, Charlotte. Đặc điểm ở 3 thành phố này cũng đều nằm trong top các thành phố có mật độ dân số cao.
    + Tai nạn hay xảy ra ở các đoạn cua, các giao lộ nhiều làn giao nhau.
2. Phụ thuộc theo thời gian  
    + Tình hình tai nạn xảy ra tăng dần theo năm, đặc biệt là năm 2020 - tăng rất nhanh so với năm 2019(gần gấp đôi).
    + Tai nạn tăng nhanh những tháng cuối năm.
    + Tai nạn thường xảy ra vào các giờ cao điểm như 7-8, 15-17.
    + Tai nạn giảm vào cuối tuần.
3. Mức độ nghiêm trọng cao có xu hướng giảm theo từng năm mặc dù số tai nạn tăng theo từng năm.
4. Tai nạn xảy ra nhiều tại các nơi thiếu các điều kiện giao thông ở gần. Điều kiện giao thông ở đây là các yếu tố sau:
    + Amenity: Cho biết có sự hiện diện của một tòa nhà họặc địa điểm nào ở gần không.
    + Bump: Cho biết có sự hiện của gờ giảm tốc ở gần hay không
    + Crossing: Cho biết có xuất hiện nơi có hai đường bộ, hai đường sắt hoặc đường bộ và đường sắt giao nhau ở gần hay không
    + Give_Way: Cho biết có các give_way(nơi nhường đường, biển báo nhường đường) ở gần không
    + Junction: Cho biết có giao lộ ở gần hay không 
    + No_Exit: Cho biết có biển báo no_exit ở gần hay không
    + Railway: Cho biết có đường sắt ở gần hay không
    + Roundabout: Cho biết có bùng binh ở gần hay không
    + Station: Cho biết có các trạm ga tàu ở gần hay không
    + Stop: Cho biết có các biển báo dừng ở gần hay không
    + Traffic_Calming: Cho biết có các biện pháp làm chậm giao thông ở gần hay không
    + Traffic_Signal: Cho biết có các tín hiểu, đèn giao thông ở gần hay không
    + Turning_Loop: Cho biết có vòng xoay ở gần hay không
5. Các điều kiện thời tiết ảnh hưởng đến tai nạn giao thông rút ra như sau:
    + Temperature(F): tai nạn xảy ra nhiều ở nhiệt độ trong đoạn [50, 70] (F) (Mức trung bình)
    + Humidity(%): phân phối khá đều, chỉ lệch ở mức độ ẩm 90% và khoảng 100% (hai điểm này tai nạn khá cao)
    + Pressure(in): Tai nạn xảy ra nhiều ở mức áp suất không khí ở khoảng 30in. Có thể cho rằng áp suất không khí không ảnh hưởng lắm vì giá trị xấp xỉ 30in là mức tiêu chuẩn ở gần mặt đất
    + Visibility(mi): Tai nạn xảy ra nhiều ở mức Visibility (khả năng nhìn thấy tính bằng m) khá thấp
    + Thời tiết đẹp thì tai nạn nhiều. Có thể suy ra rằng vào thời tiết đẹp thì mọi người ra đường nhiều nên tỉ lệ tai nạn cao
    + Wind_direction: Tai nạn xảy ra nhiều lúc trời lặng gió`
    
6. Dự đoán tương lai: Từ các phân tích `Time series` của thuộc tính `Start_Time` ở trên ta có thể dự đoán được tình hình tai nạn của nước Mỹ trong năm 2021. Nhìn vào biểu đồ có thể nhận thấy xu hướng tai nạn tăng rất nhanh vào năm 2021 với vùng màu xám thể hiện độ sai lệch của dự đoán.

## Reference

- https://plotly.com/python/bubble-maps/
- https://plotly.com/python/mapbox-density-heatmaps/
- https://plotly.com/python/choropleth-maps/
- https://phamdinhkhanh.github.io/2019/12/12/ARIMAmodel.html
- https://otexts.com/fpp2/seasonal-arima.html